In [91]:
import pandas as pd
from lxml import etree
import requests
import json

---- we can search for a place and then return first hit for a search id ----

In [92]:
#use dict to search for listings
url = "https://www.hemnet.se/locations/show"
params= {
    'q':'Ringvägen, stockholm'
}
payload={}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)



In [93]:
json_data=json.loads(response.text)
search_id=json_data[0]['id']

In [94]:
#or make dict with places to search for

#location_ids={'Ringvägen':475989,'Huvudsta':473498}
#search_id=location_ids['Ringvägen']

In [95]:
#use dict to search for listings
url = "https://www.hemnet.se/bostader"
params= {
    'housing_form_groups':'apartments',
    'location_ids':search_id,
    'item_types':'bostadsratt'
}
payload={}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)
response.url

'https://www.hemnet.se/bostader?housing_form_groups=apartments&location_ids=475989&item_types=bostadsratt'

--------- w beautiful soup ------------

In [96]:
from bs4 import BeautifulSoup

In [97]:
soup = BeautifulSoup(response.content, "html.parser")

In [98]:
#find div with good data to use
map_results=soup.find(id='results-map')
initial_data=map_results.attrs['data-initial-data']

In [99]:
#find our search key to use
json_data=json.loads(initial_data)
json_data['search_key']


'f400520c69c8fa57d345617c15f9994c8c957baf'

In [100]:
url = "https://www.hemnet.se/bostader/search/"+json_data['search_key']

params= {}

payload={}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response = requests.request("GET", url, headers=headers, data=payload, params=params)

#print(response.text)

In [101]:
r_json = response.json()
r_prop = r_json['properties']
#r_prop

In [102]:
df=pd.json_normalize(r_prop, max_level=0)
df.head(3)

,id,coordinate,address,location_name,typeSummary,iconName,project,new_construction,ongoing_bidding,foreclosure,...,labels,state_labels,user_saved,save_button_label,broker_logo_url,broker_agency_name,locations_string,should_display_showings,should_highlight_showings,trackingData
0,19226516,"[59.3188273746, 18.0518373102]",Ringvägen 9 F,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,...,"[{'category': 'feature', 'identifier': 'elevat...",[],False,Spara,https://bilder.hemnet.se/images/broker_logo_2/...,Fastighetsbyrån Stockholm - Södermalm,"Högalid, Stockholms kommun",False,False,"{'listingId': '19226516', 'listingType': 'comm..."
1,19226416,"[59.3085175, 18.0801334]",Ringvägen 135,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,...,"[{'category': 'product', 'identifier': 'premiu...","[{'category': 'state', 'identifier': 'highligh...",False,Spara,https://bilder.hemnet.se/images/broker_logo_2/...,HusmanHagberg Södermalm/Gamla Stan,"Södermalm - SoFo, Stockholms kommun",False,True,"{'listingId': '19226416', 'listingType': 'comm..."
2,19226122,"[59.3094732, 18.083821]","Ringvägen 153, 2 tr",Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,...,"[{'category': 'feature', 'identifier': 'elevat...",[],False,Spara,https://bilder.hemnet.se/images/broker_logo_2/...,Mäklarhuset Stockholm Innerstan,"Stockholm - Södermalm, Stockholms kommun",False,False,"{'listingId': '19226122', 'listingType': 'comm..."


In [106]:
#call graphql directly from listing with ids - example with one id only but can set into a loop
ql_id=df.iloc[0]['id']
ql_id

19226516

In [107]:
url_ql = "https://www.hemnet.se/graphql"

#would love to have payload_ql string formatted better but not really sure how without breaking it
payload_ql="{\"query\":\"query imagesForListing($id: ID!) {\\r\\n    listing(id: $id) \\r\\n    {  id  __typename  streetAddress  isSaved  brokerAgency \\r\\n    {    name    id    brokerCustomization \\r\\n    {      compactLogoUrl      largeLogoUrl: logoUrl(format: BROKER_CUSTOMIZATION_LARGE)      __typename    \\r\\n    }    __typename  }  ... on ActivePropertyListing \\r\\n    {    liveStreams(scope: ENDED) \\r\\n    {      embedUrl      __typename    \\r\\n    }    videoAttachment: attachment(type: VIDEO) \\r\\n    {      id      attachmentType      ... on VideoAttachment {     videoHemnetUrl        __typename     }\\r\\n          __typename    \\r\\n    }    threeDAttachment: attachment(type: THREE_D) {\\r\\n              id      ... on ThreeDAttachment {\\r\\n                          url        __typename      \\r\\n                          }      __typename    }\\r\\n                              isForeclosure    listingBrokerGalleryUrl    images(limit: 500) {\\r\\n                                        images {\\r\\n                                                    url(format: ITEMGALLERY_CUT)        portraitUrl: url(format: ITEMGALLERY_PORTRAIT_CUT)        fullscreenUrl: url(format: WIDTH1024)        originalHeight        originalWidth        labels        __typename      }      __typename    }    __typename  }  ... on ProjectUnit {    liveStreams(scope: ENDED) {      embedUrl      __typename    }    videoAttachment: attachment(type: VIDEO) {      id      attachmentType      ... on VideoAttachment {        videoHemnetUrl        __typename      }      __typename    }    threeDAttachment: attachment(type: THREE_D) {      id      ... on ThreeDAttachment {        url        __typename      }      __typename    }    isForeclosure    listingBrokerGalleryUrl    images(limit: 500) {      images {        url(format: ITEMGALLERY_CUT)        portraitUrl: url(format: ITEMGALLERY_PORTRAIT_CUT)        fullscreenUrl: url(format: WIDTH1024)        originalHeight        originalWidth        labels        __typename      }      __typename    }    __typename  }\\r\\n    }\\r\\n    }\",\"variables\":{\"id\":"+str(ql_id)+"}}"

headers_ql = {
  'Content-Type': 'application/json',
  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'
}

response_ql = requests.request("POST", url_ql, headers=headers_ql, data=payload_ql)

#print(response_ql.text)
ql_path=response_ql.json()['data']['listing']['images']['images']

ql_df=pd.json_normalize(ql_path)
#ql_df

In [112]:
#print all images from listing item
from IPython.display import Image
for index,row in ql_df.iterrows():
   display(Image(url=row['url'], width=300))